In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install python-dotenv

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## 00. Setting
노트북과 같은 폴더에 `.env`를 생성하고, 아래와 같이 `AWS_ACCESS_KEY_ID`와 `AWS_SECRET_ACCESS_KEY`를 설정합니다.

```
AWS_ACCESS_KEY_ID=your-access-key-id
AWS_SECRET_ACCESS_KEY=your-secret-access-key
```

이제 AWS Bedrock을 사용하여 LLM을 호출할 수 있습니다.

In [ ]:
import boto3
import json
from dotenv import load_dotenv

load_dotenv()

COHERE_MODEL_ID = "cohere.command-r-plus-v1:0"
PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"
PREMIER_MODEL_ID = "us.amazon.nova-premier-v1:0"

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", 
                    region_name="us-east-1"
                    )

## 01. Amazon NOVA

In [2]:
# Define your system prompt(s).
system_list = [
    { "text": "You should respond to all messages in Korean" }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [
    {"role": "user", "content": [{"text": "이순신 장군에 대해서 알려줘줘"}]},
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.9, "top_k": 20, "temperature": 0.1}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=MICRO_MODEL_ID, body=json.dumps(native_request))
request_id = response["ResponseMetadata"]["RequestId"]
print(f"Request ID: {request_id}")
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

Request ID: e9edac60-efb3-4c13-88a9-898e85f713d4

[Full Response]
{
  "output": {
    "message": {
      "content": [
        {
          "text": "\uc774\uc21c\uc2e0 \uc7a5\uad70\uc740 \uc870\uc120\uc2dc\ub300\uc758 \uc704\ub300\ud55c \ud574\uad70 \uc7a5\uad70\uc73c\ub85c, \uc784\uc9c4\uc65c\ub780 \uc2dc\uae30\uc5d0 \ud65c\uc57d\ud55c \uc778\ubb3c\uc785\ub2c8\ub2e4. \uadf8\ub294 \uc870\uc120\uc758 \ud574\uad70\uc744 \ud06c\uac8c \ubc1c\uc804\uc2dc\ucf30\uace0, \uc77c\ubcf8\uad70\uc758 \uce68\ub7b5\uc744 \ub9c9\uae30 \uc704\ud574 \ub9ce\uc740 \uc804\ud22c\uc5d0\uc11c \ud65c\uc57d\ud558\uc600\uc2b5\ub2c8\ub2e4. \ud2b9\ud788 \ud55c\uc0b0\ub3c4 \ub300\ucca9, \uba85\ub7c9 \ud574\uc804 \ub4f1\uc5d0\uc11c \ub6f0\uc5b4\ub09c \uc804\ub7b5\uacfc \uc9c0\ud718\ub825\uc744 \ubc1c\ud718\ud558\uc5ec \uc870\uc120\uc758 \ud574\uad70\uc744 \uc9c0\ucf30\uc2b5\ub2c8\ub2e4.\n\n\uc774\uc21c\uc2e0 \uc7a5\uad70\uc740 1545\ub144\uc5d0 \ud0dc\uc5b4\ub098 1598\ub144\uc5d0 \uc21c\uad6d\ud558\uc600\uc2b5\ub2c8\ub2

## 2. Cohere Command R+

In [6]:
prompt = "이순신 장군에 대해서 알려줘줘"
body = json.dumps({
    "chat_history": [],
    "message": prompt,
    "max_tokens": 512,
    "temperature": 0.1,
})

modelId = "cohere.command-r-plus-v1:0"

accept = "application/json"
contentType = "application/json"

response = client.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

result = json.loads(response.get('body').read())
print(result['text'])

이순신 장군은 조선시대 중기의 무장으로, 임진왜란 당시 조선 수군을 지휘하여 일본군에 맞서 싸운 것으로 유명한 인물입니다. 그는 뛰어난 전략과 전술로 조선 수군을 이끌며 수많은 전투에서 승리를 거두었고, 그 공로로 조선 최고의 무관인 '삼도수군통제사'에 임명되었습니다.

이순신 장군은 1545년 서울에서 태어났으며, 어린 시절부터 무예와 학문을 익히며 성장했습니다. 그는 무과 시험에 급제하여 관직에 진출한 후, 여진족과 왜구로부터 국경 지역을 방어하는 임무를 맡았습니다. 이순신 장군은 전략가이자 발명가이기도 했는데, 거북선 제작을 감독하여 조선 수군의 전력을 강화시켰습니다.

임진왜란이 발발하자, 이순신 장군은 조선 수군을 지휘하여 옥포해전, 한산도대첩, 명량대첩 등 수많은 전투에서 승리를 거두었습니다. 특히 한산도대첩에서는 학익진 전법을 사용하여 일본 수군을 크게 무찌르며 조선 수군의 위상을 드높였습니다.

이순신 장군은 군사적 업적뿐만 아니라 인품과 리더십으로도 존경을 받았습니다. 그는 부하들을 아끼고 백성들을 돌보는 데 힘썼으며, 청렴결백한 삶을 살았습니다. 그러나 정치적인 음모에 휘말려 잠시 투옥되기도 했고, 백의종군의 명령을 받고 전쟁터로 복귀하기도 했습니다.

임진왜란이 끝날 무렵인 1598년, 이순신 장군은 노량해전에서 전사했습니다. 그의 죽음은 조선과 일본 양국에 큰 충격을 주었습니다. 조선에서는 그를 기리기 위해 여러 가지 기념사업을 펼쳤고, 오늘날까지도 이순신 장군은 한국의 대표적인 영웅으로 존경받고 있습니다.
